In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Mar 10 18:56:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install transformers
!pip install peft
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.8 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/datos

Mounted at /gdrive
/gdrive/MyDrive/datos


El código carga un modelo preentrenado de clasificación de secuencias llamado "BertForSequenceClassification" del paquete "transformers". Este modelo utiliza el tokenizador "BertTokenizer" también del paquete "transformers". El modelo está preentrenado con la base "bert-base-uncased" y se espera que clasifique en 5 categorías. Finalmente, el modelo se mueve a la GPU para una mayor velocidad de procesamiento:

In [5]:
import pandas as pd
df = pd.read_excel('tweets.xlsx')
df

,name,text
0,elonmusk,Highly recommend “The Explorers” podcast for s...
1,JeffBezos,@BlueMoonBrewCo First round is on me. (2/2) an...
2,BillGates,Congratulations on a remarkable win in #India...
3,MikeBloomberg,American greatness began with immigrants. Choo...
4,MichaelDell,@sakacc @DellTech @VMware Enjoy the very well-...
...,...,...
108,sebastianpinera,Health sector: those belonging to the 40% most...
109,swimmym,In addition to the content of the choir compet...
110,yousuck2020,I bought a bookshelf in the shape of a cypress...
111,salinas,@CitlaHM @ferbelaunzaran @lopezobrador_ Cenado...


In [8]:
from transformers import RobertaForSequenceClassification
from transformers import RobertaTokenizer
import numpy as np
import random
import torch
from datasets import load_dataset
from datasets import Dataset, DatasetDict
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    LoraConfig,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
)

#configuracion previa (ajuste loRA)
model_name_or_path = "roberta-large"
peft_type = PeftType.LORA
peft_config = LoraConfig(task_type="SEQ_CLS", inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)
# Definir el nombre de las columnas del dataset
column_names = ['name', 'text']

# Convertir el dataframe en un diccionario
data = {
    column: df[column].tolist()
    for column in column_names
}

# Crear el dataset de Hugging Face
dataset = Dataset.from_dict(data)

if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"

# Tokenizar
tokenizer = RobertaTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
def tokenize_function(examples):
        outputs = tokenizer(examples["text"], truncation=True, max_length=512)
        return outputs
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["name"],
)

def collate_fn(examples):
    return tokenizer.pad(examples, padding='longest', return_tensors="pt")

# Arquitectura
device = "cuda"
num_epochs = 4
batch_size = 1
lr = 5e-5
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
num_labels=3
model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=num_labels)
model.task_name = "classification"
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model.cuda()

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,839,107 || all params: 357,201,926 || trainable%: 0.5148648050682683


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 1024, padding_idx=1)
          (position_embeddings): Embedding(514, 1024, padding_idx=1)
          (token_type_embeddings): Embedding(1, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-23): 24 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): M

In [9]:
import torch
model_path = "modelo_final_2_a.pt"
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [10]:
def get_labels():
    return ['ext', 'neu', 'opn']

def predict(model, tokenizer, text):
    # Codificar el texto
    input_ids = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True).to("cuda")

    # Obtener las probabilidades de cada clase
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs[0]
        probs = torch.softmax(logits, dim=1)

    # Obtener las probabilidades para cada clase
    class_probs = [prob.item() for prob in probs[0]]

    labels = get_labels()
    results = list(zip(labels, class_probs))

    return results

In [11]:
results = df['text'].apply(lambda x: predict(model, tokenizer, x))

In [12]:
import pandas as pd

# Crear una lista vacía para almacenar los resultados
data = []

# Iterar sobre cada fila de resultados
for i, row in results.iteritems():
    # Crear un diccionario para almacenar los resultados de esta fila
    row_data = {'name': df.loc[i, 'name']}

    # Iterar sobre cada etiqueta y probabilidad
    for label, prob in row:
        # Agregar la probabilidad al diccionario
        row_data[label] = prob

    # Agregar el diccionario a la lista de datos
    data.append(row_data)

# Crear el nuevo dataframe a partir de la lista de datos
df_rasgos_estimados = pd.DataFrame(data)


<ipython-input-12-c8172aa1ae94>:7: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, row in results.iteritems():


In [13]:
df_rasgos_estimados

,name,ext,neu,opn
0,elonmusk,0.067092,0.065018,0.867890
1,JeffBezos,0.101607,0.050289,0.848104
2,BillGates,0.356669,0.339314,0.304018
3,MikeBloomberg,0.381089,0.465839,0.153071
4,MichaelDell,0.169363,0.165265,0.665371
...,...,...,...,...
108,sebastianpinera,0.275351,0.491065,0.233584
109,swimmym,0.396436,0.223936,0.379628
110,yousuck2020,0.197205,0.332309,0.470485
111,salinas,0.302566,0.456022,0.241413


In [15]:
# Descarga el archivo
!pip install openpyxl
import openpyxl

import pandas as pd

with pd.ExcelWriter('df_rasgos_estimados.xlsx', engine='openpyxl') as writer:
    df_rasgos_estimados.to_excel(writer)

from google.colab import files
files.download('df_rasgos_estimados.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>